# Chatterbox Turbo TTS Debugging Notebook

This notebook helps debug Chatterbox Turbo TTS audio generation issues.

**Run on Google Colab with GPU enabled:**
- Runtime → Change runtime type → GPU (T4 is sufficient)

**You will need:**
- Hugging Face token with read access

## 1. Setup and Installation

In [ ]:
!pip install -q chatterbox-tts huggingface-hub torchaudio matplotlib librosa

In [ ]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Enter your Hugging Face token: ")
login(token=hf_token)
print("Authenticated with Hugging Face")

In [ ]:
import torch
import torchaudio as ta
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from IPython.display import Audio, display

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Load Chatterbox Turbo Model

In [ ]:
from chatterbox.tts_turbo import ChatterboxTurboTTS

print("Loading Chatterbox Turbo TTS model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ChatterboxTurboTTS.from_pretrained(device=device)
print(f"Model loaded on {device}")
print(f"Sample rate: {model.sr} Hz")

## 3. Helper Functions

In [ ]:
def visualize_audio(wav, sr, title="Audio Waveform"):
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    if isinstance(wav, torch.Tensor):
        wav_np = wav.cpu().numpy().squeeze()
    else:
        wav_np = wav.squeeze()
    
    axes[0].plot(wav_np)
    axes[0].set_title(f"{title} - Waveform")
    axes[0].set_xlabel("Sample")
    axes[0].set_ylabel("Amplitude")
    axes[0].grid(True, alpha=0.3)
    
    D = librosa.amplitude_to_db(np.abs(librosa.stft(wav_np)), ref=np.max)
    img = librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', ax=axes[1])
    axes[1].set_title(f"{title} - Spectrogram")
    fig.colorbar(img, ax=axes[1], format="%+2.0f dB")
    
    plt.tight_layout()
    plt.show()
    
    print(f"Duration: {len(wav_np) / sr:.2f}s")
    print(f"Shape: {wav_np.shape}")
    print(f"Amplitude: [{wav_np.min():.4f}, {wav_np.max():.4f}]")
    print(f"Mean: {wav_np.mean():.4f}, Std: {wav_np.std():.4f}")

def play_audio(wav, sr):
    if isinstance(wav, torch.Tensor):
        wav_np = wav.cpu().numpy().squeeze()
    else:
        wav_np = wav.squeeze()
    display(Audio(wav_np, rate=sr))

def save_audio(wav, sr, filename):
    ta.save(filename, wav.cpu() if isinstance(wav, torch.Tensor) else wav, sr)
    print(f"Saved to {filename}")

## 4. Test 1: Simple Text with Default Parameters

In [ ]:
text = "Hello, this is a test."
print(f"Text: {text}")
print("Using default parameters")

wav = model.generate(text)

visualize_audio(wav, model.sr, "Test 1: Default Parameters")
play_audio(wav, model.sr)
save_audio(wav, model.sr, "test1_default.wav")

## 5. Test 2: Simple Text with Your Parameters

In [ ]:
text = "Hello, this is a test."
print(f"Text: {text}")
print("Using: exaggeration=0.5, cfg_weight=0.5")

wav = model.generate(text, exaggeration=0.5, cfg_weight=0.5, temperature=1.0)

visualize_audio(wav, model.sr, "Test 2: Your Parameters")
play_audio(wav, model.sr)
save_audio(wav, model.sr, "test2_your_params.wav")

## 6. Test 3: Check Model Parameters

In [ ]:
import inspect

signature = inspect.signature(model.generate)
print("ChatterboxTurboTTS.generate() signature:")
print(signature)
print("\nParameters:")
for param_name, param in signature.parameters.items():
    default = param.default if param.default != inspect.Parameter.empty else "required"
    print(f"  {param_name}: {default}")

## 7. Test 4: Different Parameter Combinations

In [ ]:
text = "The quick brown fox jumps over the lazy dog."

tests = [
    {"name": "Default", "params": {}},
    {"name": "Low_exaggeration", "params": {"exaggeration": 0.5}},
    {"name": "High_exaggeration", "params": {"exaggeration": 1.5}},
    {"name": "Low_CFG", "params": {"cfg_weight": 0.3}},
    {"name": "High_CFG", "params": {"cfg_weight": 0.9}},
    {"name": "Your_params", "params": {"exaggeration": 0.5, "cfg_weight": 0.5}},
]

for i, test in enumerate(tests, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {test['name']}")
    print(f"Parameters: {test['params']}")
    
    wav = model.generate(text, **test['params'])
    
    filename = f"test4_{test['name']}.wav"
    save_audio(wav, model.sr, filename)
    play_audio(wav, model.sr)

## 8. Test 5: Preprocessed Text

In [ ]:
import re

def preprocess_text(text):
    lines = text.split('\n')
    paragraphs = []
    current = []
    
    for line in lines:
        cleaned = line.strip()
        if not cleaned:
            if current:
                paragraphs.append(' '.join(current))
                current = []
        else:
            current.append(cleaned)
    
    if current:
        paragraphs.append(' '.join(current))
    
    result = '\n\n'.join(paragraphs)
    return re.sub(r' +', ' ', result)

In [ ]:
raw_text = """We loitered the morning away in the garden. Stanley Hopkins had
      gone down to the village to look into some rumours of a strange
      woman who had been seen by some children on the Chatham Road."""

cleaned = preprocess_text(raw_text)

print("Raw text:")
print(repr(raw_text[:100]))
print("\nCleaned text:")
print(repr(cleaned[:100]))

In [ ]:
print("Generating with cleaned text...")
wav_clean = model.generate(cleaned, exaggeration=0.5, cfg_weight=0.5)
save_audio(wav_clean, model.sr, "test5_cleaned.wav")
play_audio(wav_clean, model.sr)

In [ ]:
print("Generating with raw text...")
wav_raw = model.generate(raw_text, exaggeration=0.5, cfg_weight=0.5)
save_audio(wav_raw, model.sr, "test5_raw.wav")
play_audio(wav_raw, model.sr)

## 9. Test 6: Short Sentences

In [ ]:
sentences = [
    "Hello.",
    "This is a test.",
    "The weather is nice today.",
]

for i, s in enumerate(sentences, 1):
    print(f"\nTest {i}: {s}")
    wav = model.generate(s, exaggeration=0.5, cfg_weight=0.5)
    save_audio(wav, model.sr, f"test6_short_{i}.wav")
    play_audio(wav, model.sr)

## 10. Diagnostic Summary

In [ ]:
print("="*60)
print("DIAGNOSTIC QUESTIONS")
print("="*60)
print("1. Does Test 1 sound good?")
print("   YES = Default parameters work")
print("   NO = Model issue")
print("")
print("2. Does Test 2 sound good?")
print("   YES = Your parameters are fine")
print("   NO = Parameters too low")
print("")
print("3. Which Test 4 sounds best?")
print("   Use those parameters!")
print("")
print("4. Test 5: Cleaned better than raw?")
print("   YES = Preprocessing helps")
print("   NO = Not a preprocessing issue")
print("")
print("5. Are Test 6 short sentences clear?")
print("   YES but long text bad = Length issue")
print("="*60)

## 11. List Generated Files

In [ ]:
import glob
import os

files = sorted(glob.glob("test*.wav"))
print(f"Generated {len(files)} test files:\n")
for f in files:
    size = os.path.getsize(f) / 1024
    print(f"  {f} ({size:.1f} KB)")
print("\nDownload from file browser on left")